In [2]:
import numpy as np
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [4]:
table_post = soup.find('table')
fields = table_post.find_all('td')

In [15]:
fields

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Manor" title="Lawrence Manor">Lawrence Manor</a>
 </td>, <td>M7A</td>, <td>

In [5]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

In [6]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M6A        North York  Lawrence Heights
4      M6A        North York    Lawrence Manor
5      M7A      Queen's Park      Not assigned
6      M9A  Downtown Toronto      Queen's Park
7      M1B       Scarborough             Rouge
8      M1B       Scarborough           Malvern
9      M3B        North York   Don Mills North

In [7]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

Postcode      Borough                                  Neighbourhood
0      M1B  Scarborough                                  Rouge,Malvern
1      M1C  Scarborough           Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough                Guildwood,Morningside,West Hill
3      M1G  Scarborough                                         Woburn
4      M1H  Scarborough                                      Cedarbrae
5      M1J  Scarborough                            Scarborough Village
6      M1K  Scarborough      East Birchmount Park,Ionview,Kennedy Park
7      M1L  Scarborough                  Clairlea,Golden Mile,Oakridge
8      M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West
9      M1N  Scarborough                     Birch Cliff,Cliffside West

In [8]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()


In [9]:
df[df['Borough'] == 'Queen\'s Park']

Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned

In [10]:
df[df['Neighbourhood'] == 'Not assigned']

Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned

In [11]:

df['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)

In [12]:
df.iloc[85]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 85, dtype: object

In [13]:
df.shape

(103, 3)

In [ ]:
df.to_csv('')

In [ ]:
import pandas as pd
link = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(link,error_bad_lines=False)


In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]